## Artificial Intelligence Assignment 3
## No3. CNN4 Batch Normalization
## 20132651 Sungjae Lee

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        ## Batch Normalization 을 수행하기 위해 batchnorm 층을 만들어 nn.BatchNorm2d() 로 생성합니다
        ## BatchNorm2d 는 3개 이상 차원의 입력에 대한 배치 정규화를 하기 위한 함수이며, 
        ## 이 때 매개변수 숫자는 앞에서 받은 입력의 개수를 작성해주면 됩니다. 
        
        self.batchnorm1 = nn.BatchNorm2d(6)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        
        ## 각각의 컨볼루션 층마다 batchnorm 을 추가하여 작성해 줍니다
        ## 이로써 각 컨볼루션 연산 이후에 배치 정규화를 진행할 수 있게 되었습니다.
        ## 굳이 컨볼루션 연산 이후에 진행하는 이유는 수업에서 말씀하셨던 것처럼 relu와 같은
        ## 활성함수 연산 이전에 배치 정규화가 이루어지는게 효과적이기 때문입니다.

        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))
        
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### (3) 화면 출력 확인 및 학습이 되고 있는지 서술

In [4]:
for epoch in range(2):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss : %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

## 기존의 최저 loss인 1.226에 비하여 Batch Norm 을 적용한 코드에서는 1.282 의 loss 가 나왔습니다.
## loss 가 늘어났다고 해서 성능이 되려 나빠졌다고 보기는 힘들지만, 성능의 향상이 없다고 보기에는 충분합니다.
## 다만 초기의 loss 감소율은 기존에 비해 꽤 빠른 편이라고 생각할 수 있겠습니다.

[1,  1000] loss : 2.224
[1,  2000] loss : 1.921
[1,  3000] loss : 1.769
[1,  4000] loss : 1.726
[1,  5000] loss : 1.643
[1,  6000] loss : 1.586
[1,  7000] loss : 1.573
[1,  8000] loss : 1.531
[1,  9000] loss : 1.574
[1, 10000] loss : 1.522
[1, 11000] loss : 1.519
[1, 12000] loss : 1.468
[2,  1000] loss : 1.426
[2,  2000] loss : 1.432
[2,  3000] loss : 1.398
[2,  4000] loss : 1.431
[2,  5000] loss : 1.385
[2,  6000] loss : 1.352
[2,  7000] loss : 1.350
[2,  8000] loss : 1.311
[2,  9000] loss : 1.379
[2, 10000] loss : 1.325
[2, 11000] loss : 1.328
[2, 12000] loss : 1.282
Finished Training


### (6) 화면 출력 확인 및 일반화 성능 서술

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images :%d %%'%(100 * correct / total))

## Accuracy 는 오히려 기존의 55% 에서 2% 감소한 53%의 결과를 보여줍니다.
## 이로 미루어 보아 성능에 큰 도움이 되지는 않는 것 같습니다.

Accuracy of the network on the 10000 test images :53 %


In [6]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

### (7) 화면 출력 확인 및 부류별 분류기의 성능 서술

In [7]:
## 이전과는 약간 다르게 deer 개체의 분류에서 상당히 낮은 결과값을 내었습니다.
## 그 외에는 car, horse, ship 이 여전히 높은 분류성능을 가지고 진행됩니다.
## 전체적인 Accuracy 는 크게 좋아지거나 나빠지지 않고 일정한 상태를 유지하고 있습니다.
## 분류기의 성능이 좋지 않은편이라고 할 수 있겠습니다.

for i in range(10):
    print('Accuracy of %5s: %2d %%'%(classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane: 55 %
Accuracy of   car: 72 %
Accuracy of  bird: 34 %
Accuracy of   cat: 54 %
Accuracy of  deer: 43 %
Accuracy of   dog: 26 %
Accuracy of  frog: 55 %
Accuracy of horse: 67 %
Accuracy of  ship: 69 %
Accuracy of truck: 58 %
